# **Modul 6 Data Mining 2020 "Recommender System"**

## **COLLABORATIVE FILTERING**

Sistem rekomendasi ***Recommender System*** adalah kumpulan algoritma yang digunakan untuk merekomendasikan item kepada pengguna berdasarkan informasi yang diambil dari pengguna. Sistem ini telah diterapkan dimana-mana anda dapat melihatnya di toko online, movies databases dan pencari kerja. Di modul ini, kita akan mengeksplorasi sistem rekomendasi berbasis konten dan mengimplementasikan versi sederhananya menggunakan Python dan pustaka Pandas.

## **Table of contents**

<div class="alert alert-block alert-info" style="margin-top: 20px">
- <a href="#ref1">Acquiring the Data</a><br />
- <a href="#ref2">Preprocessing</a><br />
- <a href="#ref3">Content-Based Filtering</a>
</div>
<br>

<a id="ref1"></a>
## **Acquiring the Data**

Untuk memperoleh dan mengekstrak data, cukup jalankan skrip Bash berikut:
Dataset diperoleh dari [GroupLens](http://grouplens.org/datasets/movielens/). Mari unduh dataset. Untuk mengunduh data, kami akan menggunakan **`!wget`**. Untuk mengunduh data, kami akan menggunakan `!wget` untuk mengunduhnya dari IBM Object Storage.

In [0]:
!wget -O moviedataset.zip https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
print('unziping ...')
!unzip -o -j moviedataset.zip 

--2020-02-16 06:38:24--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/moviedataset.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  32.4MB/s    in 4.8s    

2020-02-16 06:38:30 (31.6 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv                


Sekarang Anda siap untuk mulai bekerja dengan data tersebut!

<a id="ref2"></a>
## **Preprocessing**

Pertama, mari kita import semua yang kita perlukan:

In [0]:
#Library untuk manipulasi dataframe
import pandas as pd
#Fungsi matematika, kita hanya perlu fungsi sqrt jadi mari kita impor saja
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Sekarang mari kita baca setiap file ke dalam Dataframe:

In [0]:
# Menyimpan informasi film ke dalam bingkai data panda
movies_df = pd.read_csv ('movies.csv')
# Menyimpan informasi pengguna ke dalam bingkai data panda
ratings_df = pd.read_csv ('ratings.csv')
# head() adalah fungsi yang mendapatkan baris N pertama dari suatu bingkai data. Default N adalah 5.
movies_df.head ()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


Mari kita juga menghapus tahun dari kolom __title__ dengan menggunakan fungsi replace pandas dan menyimpannya di kolom __year__ yang baru.

In [0]:
#Menggunakan persamaan reguler untuk menemukan tahun yang disimpan di antara tanda kurung
#Kami menentukan paranthes sehingga kami tidak bertentangan dengan film yang telah bertahun-tahun dalam judulnya
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Menghapus tanda kurung
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Menghapus tahun-tahun dari kolom 'judul'
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Menerapkan fungsi strip untuk menghilangkan karakter spasi kosong yang mungkin muncul
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


Dengan itu, mari kita juga membagi nilai-nilai dalam kolom __Genres__ menjadi __list of Genres__ untuk menyederhanakan penggunaan di masa depan. Hal ini dapat dicapai dengan menerapkan fungsi string split Python pada kolom yang benar.

In [0]:
#Setiap genre dipisahkan oleh | jadi kita cukup memanggil fungsi split di |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Karena menjaga genre dalam format daftar tidak optimal untuk teknik sistem rekomendasi berbasis konten, kami akan menggunakan teknik One Hot Encoding untuk mengubah daftar genre menjadi vektor di mana setiap kolom sesuai dengan satu kemungkinan nilai fitur. Pengkodean ini diperlukan untuk memasukkan data kategori. Dalam hal ini, kami menyimpan setiap genre berbeda dalam kolom yang berisi 1 atau 0. 1 menunjukkan bahwa film memiliki genre tersebut dan 0 menunjukkan bahwa itu tidak. Mari kita juga menyimpan kerangka data ini dalam variabel lain karena genre tidak akan penting untuk sistem rekomendasi pertama kami.

In [0]:
# Menyalin kerangka data film ke yang baru karena kita tidak perlu menggunakan informasi genre dalam kasus pertama kami.
moviesWithGenres_df = movies_df.copy()

#Untuk setiap baris dalam dataframe, beralih melalui daftar genre dan tempatkan 1 ke kolom yang sesuai
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

# Mengisi nilai NaN dengan 0 untuk menunjukkan bahwa film tidak memiliki genre kolom itu
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Selanjutnya, mari kita lihat pada dataframe ratings.

In [0]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


Setiap baris dalam dataframe ratings memiliki id pengguna yang terkait dengan setidaknya satu film, peringkat, dan cap waktu yang ditampilkan ketika mereka memeriksanya. Kami tidak akan memerlukan kolom cap waktu, jadi mari letakkan untuk menghemat memori.

In [0]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<a id="ref3"></a>
## **Content-Based recommendation system**

Sekarang, mari kita lihat bagaimana menerapkan __Content-Based__ atau __Item-item Recommendation systems__. Teknik ini mencoba untuk mencari tahu apa aspek favorit pengguna dari suatu item dan kemudian merekomendasikan item yang menyajikan aspek-aspek tersebut. Dalam kasus kami, kami akan mencoba mencari tahu genre input favorit dari film dan peringkat yang diberikan.

Mari kita mulai dengan membuat pengguna input untuk merekomendasikan film ke:

Perhatikan: Untuk menambahkan lebih banyak film, cukup tambahkan jumlah elemen dalam __userInput__. Jangan ragu untuk menambahkan lebih banyak! Pastikan untuk menuliskannya dengan huruf kapital dan jika film dimulai dengan "The", seperti "The Matrix" kemudian tulis seperti ini: 'Matrix, The'.

In [0]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


### **Add movieId to input user**

Dengan input yang lengkap, mari kita ekstrak ID film input dari bingkai data film dan menambahkannya ke dalamnya.

Kita dapat mencapai ini dengan terlebih dahulu memfilter baris yang berisi judul film input dan kemudian menggabungkan subset ini dengan input dataframe. Kami juga menjatuhkan kolom yang tidak perlu untuk input untuk menghemat ruang memori.

In [0]:
#Menyaring film berdasarkan judul
inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Kemudian menggabungkannya sehingga kita bisa mendapatkan movieId. Secara implisit menggabungkannya dengan judul.
inputMovies = pd.merge(inputId, inputMovies)
#Mengumpulkan informasi yang tidak akan kami gunakan dari input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Input dataframe Final
#Jika film yang Anda tambahkan di atas tidak ada di sini, maka mungkin tidak ada dalam aslinya
#dataframe atau mungkin dieja secara berbeda, harap periksa kapitalisasi.
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


Kita akan mulai dengan mempelajari preferensi input, jadi mari kita ambil subset film yang inputnya tonton dari Dataframe yang mengandung genre yang didefinisikan dengan nilai-nilai biner.

In [0]:
#Menyaring film dari input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kita hanya perlu tabel genre aktual, jadi mari kita bersihkan ini sedikit dengan mengatur ulang indeks dan menjatuhkan film Id, judul, genre dan kolom tahun.

In [0]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sekarang kami siap untuk mulai mempelajari preferensi input!

Untuk melakukan ini, kita akan mengubah setiap genre menjadi bobot. Kita dapat melakukan ini dengan menggunakan ulasan input dan mengalikannya ke dalam tabel genre input dan kemudian merangkum tabel yang dihasilkan oleh kolom. Operasi ini sebenarnya merupakan produk titik antara matriks dan vektor, jadi kita bisa melakukannya dengan memanggil fungsi "dot" Pandas.

In [0]:
inputMovies['rating']

0    3.5
1    2.0
2    5.0
3    4.5
4    5.0
Name: rating, dtype: float64

In [0]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             10.0
Animation              8.0
Children               5.5
Comedy                13.5
Fantasy                5.5
Romance                0.0
Drama                 10.0
Action                 4.5
Crime                  5.0
Thriller               5.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 4.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Sekarang, kami memiliki bobot untuk setiap preferensi pengguna. Ini dikenal sebagai Profil Pengguna. Dengan menggunakan ini, kami dapat merekomendasikan film yang memenuhi preferensi pengguna.

Mari kita mulai dengan mengekstraksi tabel genre dari kerangka data asli:

In [0]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
genreTable.shape

(34208, 20)

Dengan profil input dan daftar lengkap film serta genre mereka, kami akan mengambil rata-rata tertimbang dari setiap film berdasarkan profil input dan merekomendasikan dua puluh film teratas yang paling memuaskan.

In [0]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.594406
2    0.293706
3    0.188811
4    0.328671
5    0.188811
dtype: float64

In [0]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
5018      0.748252
26093     0.734266
27344     0.720280
148775    0.685315
6902      0.678322
dtype: float64

## **Sekarang inilah tabel rekomendasi!**

In [0]:
#The final recommendation table
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
664,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
1824,1907,Mulan,"[Adventure, Animation, Children, Comedy, Drama...",1998
2902,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
6793,6902,Interstate 60,"[Adventure, Comedy, Drama, Fantasy, Mystery, S...",2002
8605,26093,"Wonderful World of the Brothers Grimm, The","[Adventure, Animation, Children, Comedy, Drama...",1962
8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
9296,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
9825,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
11716,51632,Atlantis: Milo's Return,"[Action, Adventure, Animation, Children, Comed...",2003


### Keuntungan dan Kekurangan Content-Based Filtering

**Keuntungan**

Pelajari preferensi pengguna
Sangat personal untuk pengguna

**Kekurangan**

Tidak memperhitungkan apa yang dipikirkan orang lain tentang item tersebut, sehingga rekomendasi item berkualitas rendah mungkin terjadi
Mengekstrak data tidak selalu intuitif
Menentukan karakteristik item apa yang tidak disukai atau disukai pengguna tidak selalu jelas